<a href="https://colab.research.google.com/github/calebddawson/IntroAI-4633-Assignment_2/blob/main/cdd370_assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2 - CNN classifier for MNIST dataset - Caleb Dawson cdd370

NOTE: Code based mainly from references given in Assignment details.

### Import Libraries

In [1]:
import numpy as np #
import pandas as pd
import io

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable

from sklearn.model_selection import train_test_split

### I uploaded the dataset locally.  The file used was train.csv gotten from https://www.kaggle.com/c/digit-recognizer/data .

Download the train.csv file and upload it using the next code block to run the rest of the program.

In [ ]:
from google.colab import files
uploaded = files.upload()

### Decode the uploaded train.csv file as a dataframe and print its dimensions.

In [3]:
df = pd.read_csv('train.csv')
print(df.shape)

(42000, 785)


### Prepare the dataset into test/train split

In [4]:
y = df['label'].values
X = df.drop(['label'],1).values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

### Fit function to run 5 epochs and print the results

In [19]:
def fit(model, train_loader, epochs):
    optimizer = torch.optim.Adam(model.parameters())
    error = nn.CrossEntropyLoss()
    model.train()
    for epoch in range(epochs):
        correct = 0
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            var_X_batch = Variable(X_batch).float()
            var_y_batch = Variable(y_batch)
            optimizer.zero_grad()
            output = model(var_X_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()

            # Total correct predictions
            predicted = torch.max(output.data, 1)[1] 
            correct += (predicted == var_y_batch).sum()
            #print(correct)
            if batch_idx % 50 == 0:
                print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch, batch_idx*len(X_batch), len(train_loader.dataset), 100.*batch_idx / len(train_loader), loss.data, float(correct*100) / float(BATCH_SIZE*(batch_idx+1))))

### Evaluate function to find and print the accuracy of a classification method

In [14]:
def evaluate(model):
#model = mlp
    correct = 0 
    for test_imgs, test_labels in test_loader:
        #print(test_imgs.shape)
        test_imgs = Variable(test_imgs).float()
        output = model(test_imgs)
        predicted = torch.max(output,1)[1]
        correct += (predicted == test_labels).sum()
    print("Test accuracy:{:.3f}% ".format( float(correct) / (len(test_loader)*BATCH_SIZE)))

## CNN

### Prepare dataset for Convolutional Nerual Network

In [15]:
BATCH_SIZE = 32

torch_X_train = torch.from_numpy(X_train).type(torch.LongTensor)
torch_y_train = torch.from_numpy(y_train).type(torch.LongTensor)

torch_X_test = torch.from_numpy(X_test).type(torch.LongTensor)
torch_y_test = torch.from_numpy(y_test).type(torch.LongTensor)

torch_X_train = torch_X_train.view(-1, 1,28,28).float()
torch_X_test = torch_X_test.view(-1,1,28,28).float()
print(torch_X_train.shape)
print(torch_X_test.shape)

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
test = torch.utils.data.TensorDataset(torch_X_test,torch_y_test)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)

torch.Size([35700, 1, 28, 28])
torch.Size([6300, 1, 28, 28])


### CNN Class with kernel size of 5x5

In [16]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5)
        self.conv3 = nn.Conv2d(32,64, kernel_size=5)
        self.fc1 = nn.Linear(3*3*64, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(F.max_pool2d(self.conv3(x),2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = x.view(-1,3*3*64 )
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
 
cnn = CNN()
print(cnn)

it = iter(train_loader)
X_batch, y_batch = next(it)
print(cnn.forward(X_batch).shape)

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)
torch.Size([32, 10])


### Run the dataset through CNN with 2 epochs

In [20]:
fit(cnn,train_loader, 2)

Epoch : 0 [0/35700 (0%)]	Loss: 0.332751	 Accuracy:93.750%
Epoch : 0 [1600/35700 (4%)]	Loss: 0.106678	 Accuracy:95.956%
Epoch : 0 [3200/35700 (9%)]	Loss: 0.096801	 Accuracy:96.071%
Epoch : 0 [4800/35700 (13%)]	Loss: 0.083452	 Accuracy:95.944%
Epoch : 0 [6400/35700 (18%)]	Loss: 0.137316	 Accuracy:95.740%
Epoch : 0 [8000/35700 (22%)]	Loss: 0.049113	 Accuracy:95.655%
Epoch : 0 [9600/35700 (27%)]	Loss: 0.145369	 Accuracy:95.660%
Epoch : 0 [11200/35700 (31%)]	Loss: 0.029279	 Accuracy:95.637%
Epoch : 0 [12800/35700 (36%)]	Loss: 0.384400	 Accuracy:95.550%
Epoch : 0 [14400/35700 (40%)]	Loss: 0.088618	 Accuracy:95.503%
Epoch : 0 [16000/35700 (45%)]	Loss: 0.244059	 Accuracy:95.553%
Epoch : 0 [17600/35700 (49%)]	Loss: 0.026463	 Accuracy:95.451%
Epoch : 0 [19200/35700 (54%)]	Loss: 0.123798	 Accuracy:95.450%
Epoch : 0 [20800/35700 (58%)]	Loss: 0.154413	 Accuracy:95.382%
Epoch : 0 [22400/35700 (63%)]	Loss: 0.062241	 Accuracy:95.435%
Epoch : 0 [24000/35700 (67%)]	Loss: 0.071908	 Accuracy:95.394%
Epoch

### Check accuracy of CNN

In [21]:
evaluate(cnn)

Test accuracy:0.954% 


### Run the dataset through CNN with 5 epochs

In [ ]:
fit(cnn,train_loader, 5)

### Check accuracy of CNN

In [ ]:
evaluate(cnn)